In [2]:
!pip install ta
#데이터셋을 만들어 주는 함수
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM,Dropout,GRU
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

df = pd.read_csv("./[경기관련소비재]현대차.csv")
df_columns = ['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.columns = df_columns
df = df.sort_values('Date')
df['Date']= df['Date'].astype('str')
from datetime import datetime
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)

ma = [5,20,60,120]
for days in ma:
    df['ma_'+str(days)] = df['Close'].rolling(window = days).mean()
    df['vma_'+str(days)] = df['Volume'].rolling(window = days).mean()

df.dropna(inplace=True)


import ta

H, L, C, V = df['High'], df['Low'], df['Close'], df['Volume']

# df['bol_high'] = ta.volatility.bollinger_hband(C)
# df['bol_low']  = ta.volatility.bollinger_lband(C)
df['MFI'] = ta.volume.money_flow_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['ADI'] = ta.volume.acc_dist_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['OBV'] = ta.volume.on_balance_volume(close=C, volume=V, fillna=True)
df['CMF'] = ta.volume.chaikin_money_flow(
    high=H, low=L, close=C, volume=V, fillna=True)

df['FI'] = ta.volume.force_index(close=C, volume=V, fillna=True)
df['EOM, EMV'] = ta.volume.ease_of_movement(
    high=H, low=L, volume=V, fillna=True)

df['VPT'] = ta.volume.volume_price_trend(close=C, volume=V, fillna=True)
df['NVI'] = ta.volume.negative_volume_index(close=C, volume=V, fillna=True)
df['VMAP'] = ta.volume.volume_weighted_average_price(
    high=H, low=L, close=C, volume=V, fillna=True)

# Volatility
df['ATR'] = ta.volatility.average_true_range(
    high=H, low=L, close=C, fillna=True)
df['BHB'] = ta.volatility.bollinger_hband(close=C, fillna=True)
df['BLB'] = ta.volatility.bollinger_lband(close=C, fillna=True)
df['KCH'] = ta.volatility.keltner_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['KCL'] = ta.volatility.keltner_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['KCM'] = ta.volatility.keltner_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['DCH'] = ta.volatility.donchian_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['DCL'] = ta.volatility.donchian_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['DCM'] = ta.volatility.donchian_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['UI'] = ta.volatility.ulcer_index(close=C, fillna=True)
# Trend
df['SMA'] = ta.trend.sma_indicator(close=C, fillna=True)
df['EMA'] = ta.trend.ema_indicator(close=C, fillna=True)
df['WMA'] = ta.trend.wma_indicator(close=C, fillna=True)
df['MACD'] = ta.trend.macd(close=C, fillna=True)
df['ADX'] = ta.trend.adx(high=H, low=L, close=C, fillna=True)
df['-VI'] = ta.trend.vortex_indicator_neg(
    high=H, low=L, close=C, fillna=True)
df['+VI'] = ta.trend.vortex_indicator_pos(
    high=H, low=L, close=C, fillna=True)
df['TRIX'] = ta.trend.trix(close=C, fillna=True)
df['MI'] = ta.trend.mass_index(high=H, low=L, fillna=True)
df['CCI'] = ta.trend.cci(high=H, low=L, close=C, fillna=True)
df['DPO'] = ta.trend.dpo(close=C, fillna=True)
df['KST'] = ta.trend.kst(close=C, fillna=True)
df['Ichimoku'] = ta.trend.ichimoku_a(high=H, low=L, fillna=True)
df['Parabolic SAR'] = ta.trend.psar_down(
    high=H, low=L, close=C, fillna=True)
df['STC'] = ta.trend.stc(close=C, fillna=True)
# Momentum
df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
df['SRSI'] = ta.momentum.stochrsi(close=C, fillna=True)
df['TSI'] = ta.momentum.tsi(close=C, fillna=True)
df['UO'] = ta.momentum.ultimate_oscillator(
    high=H, low=L, close=C, fillna=True)
df['SR'] = ta.momentum.stoch(close=C, high=H, low=L, fillna=True)
df['WR'] = ta.momentum.williams_r(high=H, low=L, close=C, fillna=True)
df['AO'] = ta.momentum.awesome_oscillator(high=H, low=L, fillna=True)
df['KAMA'] = ta.momentum.kama(close=C, fillna=True)
df['ROC'] = ta.momentum.roc(close=C, fillna=True)
df['PPO'] = ta.momentum.ppo(close=C, fillna=True)
df['PVO'] = ta.momentum.pvo(volume=V, fillna=True)


from sklearn.preprocessing import MinMaxScaler

# 피처값 스케일링
scaler = MinMaxScaler()

scaled_df = scaler.fit_transform(df.drop(['Close'], axis=1))
scaled_df = pd.DataFrame(scaled_df, columns = df.drop(['Close'], axis=1).columns)

scaler1 = MinMaxScaler()

temp = scaler1.fit_transform(df['Close'].values.reshape(-1,1))

scaled_df.insert(3, 'Close',temp)
scaled_df

df = scaled_df



import numpy as np

def make_dataset(feature, label, window_size = 20):
  
  feature_list = []
  label_list = []
  
  for i in range(len(feature) - window_size) :
    feature_list.append(feature.iloc[i:i+window_size])
    label_list.append(label.iloc[i+window_size])
  
  return np.array(feature_list), np.array(label_list)

WINDOW_SIZE=20

result = []
predict = []


####최근 20개의 데이터만
pred_feature = df[-WINDOW_SIZE :]
pred_feature = pred_feature.to_numpy()

pred_feature = pred_feature.reshape((1,20,-1))
# pred_feature.shape

import random as rn
import tensorflow as tf


for i in range(1, len(df)//1000+2):

        # 데이터 불러오기
        df_copy = df
     
        # 원하는 크기로 데이터 자르기
        df_copy = df_copy[-i*1000:]
        # 피처값, 타켓 스케일링
        
        TEST_SIZE = int(len(df_copy) * 0.7)
        
        train = df_copy[:TEST_SIZE]
        test = df_copy[TEST_SIZE:]

##200일의 데이터를 얼마나 잘 예측하느냐
        
        # feature_cols = df.drop('Close', axis =1).columns
        feature_cols = df.columns
        label_cols = ['Close']

        train_feature = train[feature_cols]
        train_label = train[label_cols]

        train_feature, train_label = make_dataset(train_feature, train_label,20)

        from sklearn.model_selection import train_test_split
        x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2, shuffle = False)
        
        
        test_feature = test[feature_cols]
        test_label = test[label_cols]
        test_feature , test_label = make_dataset(test_feature, test_label,20)
        
        
        from keras.layers.normalization import batch_normalization

        model = Sequential()
        model.add(GRU(16, 
                      input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                      return_sequences = True,
                      activation='relu'))
        model.add(GRU(16, return_sequences=False, stateful=False))
        model.add(Dense(1))
      
        
        model.compile(loss='mean_squared_error', optimizer='adam')

        early_stop = EarlyStopping(monitor='val_loss', patience=10)

        import os
        from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

        model_path = 'model'
        filename = os.path.join(model_path, 'tmp_checkpoint.h5')
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

        history = model.fit(x_train, y_train, 
                                            epochs=100, 
                                            batch_size=16,
                                            validation_data=(x_valid, y_valid), 
                                            callbacks=[early_stop, checkpoint])


        pred = model.predict(test_feature)
        # rescaleing 작업
        rescaled_y_true = scaler1.inverse_transform(np.array(test_label).reshape(-1, 1))
        rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))
        
        
        # 평가지표(RMSE) 계산
        RMSE = np.sqrt(mean_squared_error(rescaled_y_true, rescaled_pred))
        result.append(RMSE)
        print(f"{i * 1000}길이의 데이터 적용 완료\n 길이: {i * 1000}, RMSE:{RMSE}")
        print(result)
        

        new_pred = model.predict(pred_feature)
        rescaled_newpred = scaler1.inverse_transform(np.array(new_pred).reshape(-1,1))
        predict.append(rescaled_newpred)



C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
34/34 [==============================] - ETA: 0s - loss: 0.0886
Epoch 1: val_loss improved from inf to 0.06699, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 31s 273ms/step - loss: 0.0886 - val_loss: 0.0670
Epoch 2/100
34/34 [==============================] - ETA: 0s - loss: 0.0071
Epoch 2: val_loss improved from 0.06699 to 0.05805, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 2s 59ms/step - loss: 0.0071 - val_loss: 0.0580
Epoch 3/100
33/34 [============================>.] - ETA: 0s - loss: 0.0041
Epoch 3: val_loss did not improve from 0.05805
34/34 [==============================] - 2s 51ms/step - loss: 0.0041 - val_loss: 0.0651
Epoch 4/100
33/34 [============================>.] - ETA: 0s - loss: 0.0028
Epoch 4: val_loss did not improve from 0.05805
34/34 [==============================] - 2s 54ms/step - loss: 0.0028 - val_loss: 0.0601
Epoch 5/100
34/34 [==============================] - ETA: 0s - loss:

33/34 [============================>.] - ETA: 0s - loss: 3.0530e-04
Epoch 35: val_loss improved from 0.00171 to 0.00137, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 2s 51ms/step - loss: 3.0510e-04 - val_loss: 0.0014
Epoch 36/100
34/34 [==============================] - ETA: 0s - loss: 3.0221e-04
Epoch 36: val_loss did not improve from 0.00137
34/34 [==============================] - 2s 57ms/step - loss: 3.0221e-04 - val_loss: 0.0019
Epoch 37/100
34/34 [==============================] - ETA: 0s - loss: 2.5854e-04
Epoch 37: val_loss did not improve from 0.00137
34/34 [==============================] - 2s 54ms/step - loss: 2.5854e-04 - val_loss: 0.0016
Epoch 38/100
34/34 [==============================] - ETA: 0s - loss: 2.5264e-04
Epoch 38: val_loss improved from 0.00137 to 0.00126, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 2s 72ms/step - loss: 2.5264e-04 - val_loss: 0.0013
Epoch 39/100
34/34 [===================

34/34 [==============================] - ETA: 0s - loss: 2.1176e-04
Epoch 69: val_loss did not improve from 0.00074
34/34 [==============================] - 2s 61ms/step - loss: 2.1176e-04 - val_loss: 9.8196e-04
Epoch 70/100
33/34 [============================>.] - ETA: 0s - loss: 1.7480e-04
Epoch 70: val_loss did not improve from 0.00074
34/34 [==============================] - 2s 58ms/step - loss: 1.7937e-04 - val_loss: 7.3837e-04
1000길이의 데이터 적용 완료
 길이: 1000, RMSE:8989.714517578292
[8989.714517578292]
Epoch 1/100
68/69 [============================>.] - ETA: 0s - loss: 0.0293
Epoch 1: val_loss improved from inf to 0.00093, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 39s 136ms/step - loss: 0.0290 - val_loss: 9.2849e-04
Epoch 2/100
68/69 [============================>.] - ETA: 0s - loss: 0.0013
Epoch 2: val_loss improved from 0.00093 to 0.00067, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 4s 57ms/step - loss: 0.0

69/69 [==============================] - 4s 58ms/step - loss: 2.1107e-04 - val_loss: 1.8136e-04
Epoch 32/100
69/69 [==============================] - ETA: 0s - loss: 2.1038e-04
Epoch 32: val_loss improved from 0.00018 to 0.00018, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 3s 43ms/step - loss: 2.1038e-04 - val_loss: 1.7660e-04
Epoch 33/100
69/69 [==============================] - ETA: 0s - loss: 2.0464e-04
Epoch 33: val_loss improved from 0.00018 to 0.00017, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 4s 52ms/step - loss: 2.0464e-04 - val_loss: 1.6874e-04
Epoch 34/100
67/69 [============================>.] - ETA: 0s - loss: 2.0804e-04
Epoch 34: val_loss did not improve from 0.00017
69/69 [==============================] - 3s 47ms/step - loss: 2.0475e-04 - val_loss: 1.8125e-04
Epoch 35/100
69/69 [==============================] - ETA: 0s - loss: 2.1382e-04
Epoch 35: val_loss did not improve from 0.00017
69/69 [===

104/104 [==============================] - 9s 85ms/step - loss: 3.0541e-04 - val_loss: 1.8612e-04
Epoch 26/100
103/104 [============================>.] - ETA: 0s - loss: 3.0073e-04
Epoch 42: val_loss did not improve from 0.00014
104/104 [==============================] - 6s 54ms/step - loss: 3.0047e-04 - val_loss: 1.7428e-04
Epoch 43/100
103/104 [============================>.] - ETA: 0s - loss: 2.9794e-04
Epoch 43: val_loss did not improve from 0.00014
104/104 [==============================] - 5s 45ms/step - loss: 2.9679e-04 - val_loss: 2.7650e-04
Epoch 44/100
102/104 [============================>.] - ETA: 0s - loss: 3.7184e-04
Epoch 44: val_loss improved from 0.00014 to 0.00013, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 6s 57ms/step - loss: 3.7675e-04 - val_loss: 1.2888e-04
Epoch 45/100
102/104 [============================>.] - ETA: 0s - loss: 2.6981e-04
Epoch 45: val_loss did not improve from 0.00013
104/104 [==============================

Epoch 76/100
104/104 [==============================] - ETA: 0s - loss: 2.6213e-04
Epoch 76: val_loss did not improve from 0.00012
104/104 [==============================] - 5s 44ms/step - loss: 2.6213e-04 - val_loss: 1.3263e-04
Epoch 77/100
104/104 [==============================] - ETA: 0s - loss: 2.4454e-04
Epoch 77: val_loss did not improve from 0.00012
104/104 [==============================] - 4s 41ms/step - loss: 2.4454e-04 - val_loss: 1.5848e-04
Epoch 78/100
138/139 [============================>.] - ETA: 0s - loss: 1.9981e-04
Epoch 41: val_loss did not improve from 0.00014
139/139 [==============================] - 8s 55ms/step - loss: 1.9987e-04 - val_loss: 3.3987e-04
Epoch 42/100
137/139 [============================>.] - ETA: 0s - loss: 2.5097e-04
Epoch 42: val_loss did not improve from 0.00014
139/139 [==============================] - 6s 42ms/step - loss: 2.4953e-04 - val_loss: 2.3821e-04
Epoch 43/100
138/139 [============================>.] - ETA: 0s - loss: 2.4610e-04
E

Epoch 26/100
173/174 [============================>.] - ETA: 0s - loss: 1.5070e-04
Epoch 26: val_loss did not improve from 0.00022
174/174 [==============================] - 9s 55ms/step - loss: 1.5304e-04 - val_loss: 2.2061e-04
Epoch 27/100
173/174 [============================>.] - ETA: 0s - loss: 1.7374e-04
Epoch 27: val_loss did not improve from 0.00022
174/174 [==============================] - 8s 47ms/step - loss: 1.7487e-04 - val_loss: 3.1465e-04
Epoch 28/100
174/174 [==============================] - ETA: 0s - loss: 1.7222e-04
Epoch 28: val_loss did not improve from 0.00022
174/174 [==============================] - 8s 48ms/step - loss: 1.7222e-04 - val_loss: 5.3376e-04
Epoch 29/100
174/174 [==============================] - ETA: 0s - loss: 1.6515e-04
Epoch 29: val_loss improved from 0.00022 to 0.00022, saving model to model\tmp_checkpoint.h5
174/174 [==============================] - 9s 49ms/step - loss: 1.6515e-04 - val_loss: 2.1705e-04
Epoch 30/100
173/174 [=================

244/244 [==============================] - ETA: 0s - loss: 6.0016e-05
Epoch 12: val_loss improved from 0.02295 to 0.02209, saving model to model\tmp_checkpoint.h5
244/244 [==============================] - 16s 67ms/step - loss: 6.0016e-05 - val_loss: 0.0221
Epoch 13/100
244/244 [==============================] - ETA: 0s - loss: 3.7643e-05
Epoch 33: val_loss improved from 0.01022 to 0.00894, saving model to model\tmp_checkpoint.h5
244/244 [==============================] - 7s 30ms/step - loss: 3.7643e-05 - val_loss: 0.0089
Epoch 34/100
243/244 [============================>.] - ETA: 0s - loss: 3.2680e-05
Epoch 34: val_loss improved from 0.00894 to 0.00878, saving model to model\tmp_checkpoint.h5
244/244 [==============================] - 8s 32ms/step - loss: 3.2593e-05 - val_loss: 0.0088
Epoch 35/100
242/244 [============================>.] - ETA: 0s - loss: 3.7222e-05
Epoch 35: val_loss improved from 0.00878 to 0.00866, saving model to model\tmp_checkpoint.h5
244/244 [=================

Epoch 3/100
279/279 [==============================] - ETA: 0s - loss: 1.7487e-04
Epoch 3: val_loss did not improve from 0.00235
279/279 [==============================] - 8s 29ms/step - loss: 1.7487e-04 - val_loss: 0.0072
Epoch 4/100
278/279 [============================>.] - ETA: 0s - loss: 1.2357e-04
Epoch 4: val_loss did not improve from 0.00235
279/279 [==============================] - 11s 39ms/step - loss: 1.2330e-04 - val_loss: 0.0121
Epoch 5/100
277/279 [============================>.] - ETA: 0s - loss: 8.7101e-05
Epoch 5: val_loss did not improve from 0.00235
279/279 [==============================] - 6s 22ms/step - loss: 8.6952e-05 - val_loss: 0.0113
Epoch 6/100
277/279 [============================>.] - ETA: 0s - loss: 6.6857e-05
Epoch 6: val_loss did not improve from 0.00235
279/279 [==============================] - 9s 31ms/step - loss: 6.6618e-05 - val_loss: 0.0106
Epoch 7/100
279/279 [==============================] - ETA: 0s - loss: 5.2683e-05
Epoch 7: val_loss did not

In [3]:
print(predict)

[array([[179978.98]], dtype=float32), array([[186548.73]], dtype=float32), array([[191460.25]], dtype=float32), array([[193515.19]], dtype=float32), array([[193878.69]], dtype=float32), array([[193060.84]], dtype=float32), array([[184288.52]], dtype=float32), array([[169814.44]], dtype=float32), array([[144873.81]], dtype=float32), array([[122595.86]], dtype=float32)]
